## Importing Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

## Load env file

In [2]:
## load env
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
load_dotenv(os.path.join(BASE_DIR, '.env'))

True

## Retrieving data from Sitemap

In [4]:
base_url = os.getenv("BASE_URL")
request_url = f"{base_url}/GhanaHomePage/sitemap.php"
page = requests.get(request_url)
# print(page.text)

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
results = soup.find(id = "medsection2")

In [8]:
# print(results.prettify())

In [9]:
site_map_data = results.find_all("dl", class_ = "resources")

In [10]:
df_columns = ['Section','url']
sections_df = pd.DataFrame(columns = df_columns)

In [11]:
for sections in site_map_data:
    urls = sections.find_all("a")
    for url in urls:
        temp_row = pd.DataFrame(data = [[url.text,base_url+url.get('href')]], columns = df_columns)
        sections_df = pd.concat([sections_df,temp_row], ignore_index=True)

## Selecting Story Sections with articles

In [27]:
section_list = ["Abroad, Ghanaians","AFCON","African News","Athletics",
                "BBC Hausa News", "BBC Pidgin News","Business & Economy",
                "Coronavirus","Crime News","Editorial News","Entertainment",
                "Health News","Lifestyle","Music","Political News",
                "Regional News","Sports Section","Tabloid News"]

In [28]:
# getting the urls to the different sectiions
def get_section_urls(row):
    df = pd.DataFrame()
    if row['Section'] in section_list:
        temp_row = pd.DataFrame(data = [[row["Section"], row["url"]]], columns = df_columns)
        df = pd.concat([df,temp_row],ignore_index =True)
    else:
        pass
    return df

In [29]:
df = sections_df.apply(lambda row : get_section_urls(row),axis = 1)

In [38]:
df_columns = ['Section','url']
stories_urls_df = pd.DataFrame(columns = df_columns)

In [39]:
for row in df:
    stories_urls_df = pd.concat([stories_urls_df,row])

In [40]:
stories_urls_df.reset_index(inplace=True,drop=True)

In [46]:
stories_urls_df.to_csv("C:\\Users\\annieboadu\\Documents\\ghana-web-data-main\\csv_files\\selected_sections_data.csv")